In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def jac_to_ic(rg,rp,theta,m1,m2,m3):
    
    thrad=np.radians(theta)

    mred2 = m2/(m2+m3)
    mred3 = m3/(m2+m3)

    r12=np.sqrt(rg*rg + mred3*mred3*rp*rp + 2.0*rg*mred3*rp*np.cos(thrad))
    r13=np.sqrt(rg*rg + mred2*mred2*rp*rp - 2.0*rg*mred2*rp*np.cos(thrad))
    r23=rp

    return r12,r13,r23

def ic_to_jac(r12,r13,r23,m1,m2,m3):

    mred2 = m2/(m1+m2)
    d1 = mred2*r13
    cbeta = (r12*r12+r13*r13-r23*r23)/(2.0*r12*r13)

    rp = r13
    rg = np.sqrt(r12*r12+d1*d1-2.0*r12*d1*cbeta)
    ctheta = (r12*r12-d1*d1-rg*rg)/(-2.0*rg*d1)
    theta = np.degrees(np.arccos(ctheta))

    return rg,rp,theta

def transform_coords(rg,rp,theta,m1,m2,m3):

    r12,r13,r23=jac_to_ic(rg,rp,theta,m1,m2,m3)
    rgp,rpp,thetap=ic_to_jac(r12,r13,r23,m1,m2,m3)

    return rgp,rpp,thetap

In [3]:
print(transform_coords(2.65,1.4,90,30.9737620,2.0141017778,2.0141017778))

(np.float64(2.5966922514026716), np.float64(2.7408940147331493), np.float64(148.58264539291366))


In [4]:
path_ex = "/home/jorgebdelafuente/Doctorado/Quantics/PH2p_1TApp/exact_test/"
rpgrid = np.loadtxt(path_ex+"rpgrid.dat",skiprows=11)[:,1]
rggrid = np.loadtxt(path_ex+"rggrid.dat",skiprows=11)[:,1]
thgrid = np.loadtxt(path_ex+"thgrid.dat",skiprows=11)[:,1]
print(rpgrid)

[0.5        0.52349869 0.54699739 0.57049608 0.59399478 0.61749347
 0.64099217 0.66449086 0.68798956 0.71148825 0.73498695 0.75848564
 0.78198433 0.80548303 0.82898172 0.85248042 0.87597911 0.89947781
 0.9229765  0.9464752  0.96997389 0.99347258 1.01697128 1.04046997
 1.06396867 1.08746736 1.11096606 1.13446475 1.15796345 1.18146214
 1.20496084 1.22845953 1.25195822 1.27545692 1.29895561 1.32245431
 1.345953   1.3694517  1.39295039 1.41644909 1.43994778 1.46344648
 1.48694517 1.51044386 1.53394256 1.55744125 1.58093995 1.60443864
 1.62793734 1.65143603 1.67493473 1.69843342 1.72193211 1.74543081
 1.7689295  1.7924282  1.81592689 1.83942559 1.86292428 1.88642298
 1.90992167 1.93342037 1.95691906 1.98041775 2.00391645 2.02741514
 2.05091384 2.07441253 2.09791123 2.12140992 2.14490862 2.16840731
 2.19190601 2.2154047  2.23890339 2.26240209 2.28590078 2.30939948
 2.33289817 2.35639687 2.37989556 2.40339426 2.42689295 2.45039164
 2.47389034 2.49738903 2.52088773 2.54438642 2.56788512 2.5913

In [5]:
maxgrid = rggrid.shape[0]*rpgrid.shape[0]*thgrid.shape[0]

old_grid = np.zeros((maxgrid,3))
new_grid = np.zeros((maxgrid,3))

idx=0
for ith,th in enumerate(thgrid):
    for ig,rg in enumerate(rggrid):
        for ip,rp in enumerate(rpgrid):
            idx = ip + (ig)*rpgrid.shape[0] + (ith)*rpgrid.shape[0]*rggrid.shape[0]
            # print(idx,ip,ig,ith)
            rgp,rpp,thetap = transform_coords(rg,rp,th,30.9737620,2.0141017778,2.0141017778)
            old_grid[idx,0], old_grid[idx,1], old_grid[idx,2] = rp,rg,th
            new_grid[idx,0], new_grid[idx,1], new_grid[idx,2] = rpp,rgp,thetap

rppmin = new_grid[:,0].min()
rppmax = new_grid[:,0].max()
rgpmin = new_grid[:,1].min()
rgpmax = new_grid[:,1].max()
thetamin = new_grid[:,2].min()
thetamax = new_grid[:,2].max()
# print(rppmin,rppmax,rgpmin,rgpmax,thetamin,thetamax)


In [28]:
#FFT dvr New Grid
dvr_rp = np.linspace(rppmin,rppmax,rpgrid.shape[0])
dvr_rg = np.linspace(rgpmin,rgpmax,rggrid.shape[0])
dvr_theta = np.linspace(thetamin,thetamax,thgrid.shape[0])

thrhld = 1.e-2
counter = 0
for ith, th in enumerate(dvr_theta):
    for ig, rg in enumerate(dvr_rg):
        for ip, rp in enumerate(dvr_rp):
            idx = ip + (ig)*dvr_rp.shape[0] + (ith)*dvr_rp.shape[0]*dvr_rg.shape[0]
            diffrp = abs(rp - new_grid[idx,0])
            diffrg = abs(rg - new_grid[idx,1])
            diffrtheta = abs(th - new_grid[idx,2])
            if abs(diffrp) < thrhld or abs(diffrg) < thrhld or abs(diffrtheta) < thrhld:
                counter = counter + 1

print(counter)

99150
